+ 김형신 교수님의 엠비언트 인공지능 수업 참고

# 1. Tensorflow Input data handling
 Tensorflow의 Data 관련한 API들에 관하여 소개할 것이다. 관련 API가 여러 가지가 있지만 실습에서는 아래 API를 다룰 것이다.
- Keras data API
- Keras preprocess API
- Tensorflow data API

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

## [1] Keras data API
먼저 Keras에서 제공하는 간단한 data API들을 소개한다.



### (1) Keras datasets
많이 사용되는 작은 데이터셋들을 사용하기 편하게 아래와 같은 모듈 형태로 제공하고, load_data()를 통해 numpy array로 받아올 수 있다. 아래의 데이터셋들을 사용할 수 있다.

* boston_housing module: Boston housing price regression dataset.
* cifar10 module: CIFAR10 small images classification dataset.
* cifar100 module: CIFAR100 small images classification dataset.
* fashion_mnist module: Fashion-MNIST dataset.
* imdb module: IMDB sentiment classification dataset.
* mnist module: MNIST handwritten digits dataset.
* reuters module: Reuters topic classification dataset.

In [8]:
fashion_mnist = tf.keras.datasets.cifar10

#Load data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_train[1].shape)

#Split train| set into train/valid set
from sklearn import model_selection
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train, y_train,test_size=0.2)

(50000, 32, 32, 3) (50000, 1)
(32, 32, 3)


### (2) Data preprocessing/agumentation using Keras layers
**현재 서버에는 Tensorflow 2.1버전이 설치되어 있어 사용불가** <br>
**별도로 최신 버전의 Tensorflow 설치시 사용가능**<br><br>
최신 버전의 keras에서는 간단한 data preprocessing/augmentation을 할 수 있도록 layer를 생성하는 기능이 추가되었다. 

- `tf.keras.layers.experimental.preprocessing`: Rescaling(변수 스케일링), RandomFlip(상하/좌우 반전), RandomRotation(회전) 등의 data augmentation을 적용 가능하다. 더욱 자세한 내용은 아래 링크를 참조하자.<br> https://keras.io/guides/preprocessing_layers/#:~:text=Keras%20preprocessing%20layers%20The%20Keras%20preprocessing%20layers%20API,and%20exported%20as%20part%20of%20a%20Keras%20SavedModel.

실습을 통해 활용해 보도록 하자. 아래 작업을 해볼 것이다.<br>
- 0 ~ 255사이의 정수로 들어온 데이터를 0~1사이의 실수 값으로 변환
- 상하/좌우 랜덤 반전
- 회전(2pi * 20%)


#### (a) 활용법 1
다른 Layer들과 마찬가지로 Model에 layer를 추가해주면 네트워크가 시작하기 전에 전처리 등을 수행하게 된다.

In [10]:
# augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255.0),   # 255로 나눠서, 0 ~ 1 사이 값으로 만들기
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)

])

In [11]:
model = tf.keras.models.Sequential([  
  data_augmentation,     # agumentation
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", input_shape=(28,28,1)),
  tf.keras.layers.MaxPool2D((2,2)),
  tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
  tf.keras.layers.MaxPool2D((2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax') 
])

#### (b) 활용법 2
또는 모델에는 포함시키지 않고 아래와 같은 방식으로 Data가 가공되도록 구성할 수도 있다.

In [ ]:
#Data preprocessing/augmentation for training images
x_train = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x_train)
x_train = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")(x_train)
x_train = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x_train)

#Data preprocessing/augmentation for validation/test images
x_valid = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x_valid)
x_test = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x_valid)